# Notebook for training/testing MLP network

We train a set of `MLP` networks on different datasets (**for different N in system, N features uses - fixed**)

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12, 8)

In [ ]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()    

data_path = p_join('..', '..', 'data')
N_FEAT = 16                  # Set n_feat. parameter - uses for count central features
#################################
####### GLOBAL CONFIG ###########
#################################
CONFIG = {'ephs': 100,
          'train_batch_size': 1024,
          'test_batch_size': 64,
          'cpu_workers': 2,
          'save_ckpts': False,
          'ckpt_save_folder': f'mlp_ckpts_feat_{N_FEAT}'}

In [ ]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utils import utils
from utils import dl_models

### Run training

In [ ]:
### Each subarray represent an experiment,
### where N will be used for dataset creation 
N_SLICES = [[21, 22], [23, 24], [25, 26], [27, 28]]

In [ ]:
results = {}
for nums in N_SLICES:
    n_global_str = '-'.join(map(str, nums))
    CONFIG['ckpt_save_folder'] = f'mlp_ckpts_N_{n_global_str}_feat_{N_FEAT}'
    CONFIG['experiment_name'] = f'N_C={N_FEAT}, N={n_global_str}'
    print(f"Starting {CONFIG['experiment_name']} ...")

    ################################
    ####    Create dataset      ####
    ################################
    X, Y = utils.make_merged_dataset_by_N(data_path, nums, n_feat=N_FEAT)
    train_dataloader, test_dataloader = utils.create_dataloaders(
                                            X, Y,
                                            cpu_workers=CONFIG['cpu_workers'],
                                            train_bs=CONFIG['train_batch_size'],
                                            test_bs=CONFIG['test_batch_size']
                                        )


    ################################
    ####    Run training        ####
    ################################
    net = dl_models.MlpNetLight(in_features=N_FEAT)
    utils.count_params(net)
    optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 5e-3))
    score = utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)
    results[CONFIG['experiment_name']] = {'Accuracy': score}

In [ ]:
results